In [ ]:
import os
import sys

dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import io
import time

import paramiko


from utils.config_utils import find_root_dir

from private_keys import KNUCKLES_PASSWORD, BLAZE_PRIVATE_KEY, GITHUB_PRIVATE_KEY

In [ ]:
SCRIPT_TO_RUN = 'run_experiment_test.sh'  # 'run_experiment_1.sh', 'run_experiment_2.sh', etc.

## Connecting to Blaze

In [ ]:
# Private key
k = BLAZE_PRIVATE_KEY.strip()
padding = len(k) % 4
if padding > 0:
    k += "=" * (4 - padding)
private_key_file = io.StringIO(k)
pkey = paramiko.RSAKey.from_private_key(private_key_file)

# Establish SSH connection to the first server (knuckles)
knuckles = paramiko.SSHClient()
knuckles.set_missing_host_key_policy(paramiko.AutoAddPolicy())
knuckles.connect('knuckles.cs.ucl.ac.uk', username='amarques', password=KNUCKLES_PASSWORD)

# Use this connection to establish a second SSH connection to the second server (blaze)
# Paramiko Transport is used here for nested ssh
transport = knuckles.get_transport()
dest_addr = ('blaze.cs.ucl.ac.uk', 22) # IP and port
local_addr = ('knuckles.cs.ucl.ac.uk', 22) # IP and port
jump = transport.open_channel("direct-tcpip", dest_addr, local_addr)

blaze = paramiko.SSHClient()
blaze.set_missing_host_key_policy(paramiko.AutoAddPolicy())
blaze.connect('blaze.cs.ucl.ac.uk', username='amarques', sock=jump, pkey=pkey)

## Set up environment, code, files

In [ ]:
ardt_dir = find_root_dir()

In [ ]:
# Run commands on the second server (blaze)
blaze.exec_command(f'rm -rf /cs/student/msc/dsml/2022/amarques/action-robust-decision-transformer')
blaze.exec_command(f'git clone https://{GITHUB_PRIVATE_KEY}@github.com/afonsosamarques/action-robust-decision-transformer.git')

blaze.exec_command('source /usr/local/cuda/CUDA_VISIBILITY.csh')
blaze.exec_command('source /opt/Python/Python-3.10.1_Setup.csh')

blaze.exec_command('mkdir ardt-env')
blaze.exec_command('python3.10 -m venv ./ardt-env/ardt')
blaze.exec_command('source /cs/student/msc/dsml/2022/amarques/ardt-env/ardt/bin/activate.csh')

blaze.exec_command('pip3 install --upgrade pip')
blaze.exec_command('pip3 install -r /cs/student/msc/dsml/2022/amarques/action-robust-decision-transformer/requirements.txt')

doc = f'{ardt_dir}/access_tokens.py'
os.system(f"rsync -avz {doc} amarques@knuckles.cs.ucl.ac.uk:/cs/student/msc/dsml/2022/amarques")
blaze.exec_command('mv ./access_tokens.py /cs/student/msc/dsml/2022/amarques/action-robust-decision-transformer/ardt')

doc = f'{ardt_dir}/all-notebooks/clusters-nbs/other-run-scripts/{SCRIPT_TO_RUN}'
os.system(f"rsync -avz {doc} amarques@knuckles.cs.ucl.ac.uk:/cs/student/msc/dsml/2022/amarques")
blaze.exec_command(f'chmod +x {SCRIPT_TO_RUN}')

doc = f'{ardt_dir}/ardt/datasets'
os.system(f"rsync -avz {doc} amarques@knuckles.cs.ucl.ac.uk:/cs/student/msc/dsml/2022/amarques")

blaze.exec_command('mkdir eval-outputs')
blaze.exec_command('mkdir eval-outputs-pipeline')
blaze.exec_command('mkdir eval-outputs-test')
blaze.exec_command('mkdir agents')
blaze.exec_command('mkdir agents-pipeline')
blaze.exec_command('mkdir agents-test')
blaze.exec_command('mkdir wandb')
blaze.exec_command('mkdir wandb-json')

## Run script and close connections

In [ ]:
blaze.exec_command("/cs/student/msc/dsml/2022/amarques/" + SCRIPT_TO_RUN)

In [ ]:
time.sleep(60)
blaze.close()
knuckles.close()